# DL2 - Sheet 02: Attention (50 points)



In [50]:
# Install the transformers library if necessary
#pip install transformers


# Intro: Transformers for Sequence Classification

In this weeks notebook, we will implement our own Transformer model from scratch. Typical Transformers can be broken down into the following components:
Remark: Here, we will focus on the encoding of sentences for the purpose of sentiment classification, the decoder used in sequence2sequences Transformers has a very similar structure.



1.   Embedding: An embedding layer that transforms word tokens into vector representations.
2.  Encoder: The encoder block consists of several multi-headed attention blocks

    2.1.   Attention block 1

    2.2.   Attention block 2

    ...

    2.L.   Attention block L

3. Pooling: The final output of the encoder computes one vector representation for each token. To further summarize/pool this information for classification, the [CLS] token at sequence position 0 is typically selected.

4. Classification: A standard small MLP is used for classification and outputs probabilites for the most likely predicted class.


## Preparation: From sentences to tokens

First, we will use hugginface's tokenizer to go from words to indices in the vocabulary. In the following, we will focus on the distilBERT model:


In [1]:
import transformers

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "This was one of the best movies I have ever seen."
inputs = tokenizer(sentence, return_tensors = 'pt')
print(inputs)

{'input_ids': tensor([[ 101, 2023, 2001, 2028, 1997, 1996, 2190, 5691, 1045, 2031, 2412, 2464,
         1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [3]:
# Now we download pretrained weights for the model.
! wget https://tubcloud.tu-berlin.de/s/GfEoq4r8Sgb2727/download/distilbert.pt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# This config contains the model parameters, it will be important to understand what representations 
# have which dimensionality

from torch import nn
import torch
import math
import torch

class Config(object):
    def __init__(self):
        self.n_heads = 12
        self.n_layers = 6
        self.pad_token_id = 0
        self.dim = 768
        self.hidden_dim = 3072
        self.max_position_embeddings = 512
        self.vocab_size = 30522
        self.eps = 1e-12

        self.attention_head_size = int(self.dim / self.n_heads)
        self.all_head_size = self.n_heads * self.attention_head_size

        self.n_classes = 2        
        self.device = 'cpu'

config = Config()

# 1. Embedding Layer (5p)
Next, we will have to implement the Embedding layer. 


1.   forward: compute the output embeddings from the input_embeds and position_embeds. (Think about how they are merged.)




In [37]:
torch.manual_seed(0) # set seed for reproducible random initialization of weights

class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.dim, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.dim)
        self.LayerNorm = nn.LayerNorm(config.dim, eps=config.eps)

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        Parameters:
            input_ids (torch.Tensor):
                torch.tensor(bs, max_seq_length) The token ids to embed.
        Returns: torch.tensor(bs, max_seq_length, dim) The embedded tokens (plus position embeddings)
        """
 
        # Embedding the input ids
        input_embeds = self.word_embeddings(input_ids)  # (bs, max_seq_length, dim)
        seq_length = input_embeds.size(1)

        # Creating and embedding the position ids  
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)  # (max_seq_length)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)  # (bs, max_seq_length)
        position_embeddings = self.position_embeddings(position_ids)  # (bs, max_seq_length, dim)
        
        # 1. START YOUR CODE HERE #
        embeddings = position_embeddings + input_embeds
        # 1. END YOUR CODE HERE #
    
        embeddings = self.LayerNorm(embeddings)  # (bs, max_seq_length, dim)
        return embeddings

embedding_layer = Embeddings(config)
# Test if your embedding layer computes an output
embeddings = embedding_layer(inputs['input_ids'])
embeddings

tensor([[[ 0.7850, -0.1792, -0.7408,  ..., -0.3442, -0.2749, -0.4575],
         [ 0.3348, -0.6993,  0.9561,  ...,  0.2761, -1.1587,  0.7851],
         [ 0.2848, -0.3849,  1.1479,  ...,  0.0363, -0.7349,  1.1516],
         ...,
         [ 0.0217, -0.4367,  0.5442,  ...,  0.3884, -0.7516,  0.6763],
         [ 0.6787,  0.4753,  0.0463,  ...,  1.3417, -1.0327,  1.0803],
         [-0.2996,  0.0238,  0.8504,  ..., -1.1545, -0.5847, -0.9615]]],
       grad_fn=<NativeLayerNormBackward0>)

# 2. Attention Block (20 points)
The next step is writing the attention block. It mainly consits of the self-attention function (that you have analyzed in the first part of the exercise sheet), a layer normalization followed by an additional projection layer.

Please add the missing code:


1.   \_\_init\_\_: Add the Linear projections for the query, key and value functions. Make sure your set the correct dimensions.
2.   forward: Write the main self-attention function. Follow the three main steps as indicated in the comments below.




In [49]:
torch.manual_seed(0)
   
class AttentionBlock(nn.Module):    
    def __init__(self, config):
        super().__init__()
        self.config = config

        # self-attention components
        
        # 1. START YOUR CODE HERE #
        # Linear layers for q, k, v
        self.q_lin = nn.Linear(config.dim, config.all_head_size)
        self.k_lin = nn.Linear(config.dim, config.all_head_size)
        self.v_lin = nn.Linear(config.dim, config.all_head_size)
        
        # 1. END YOUR CODE HERE #
        
        self.out_lin = nn.Linear(in_features=config.dim, out_features=config.dim, bias=True)
        self.sa_layer_norm = nn.LayerNorm(normalized_shape=config.dim, eps=config.eps) 
        
        # feed-forward network
        self.lin1 = nn.Linear(in_features=config.dim, out_features=3072, bias=True)
        self.lin2 = nn.Linear(in_features=3072, out_features=config.dim, bias=True)
        self.output_layer_norm = nn.LayerNorm(normalized_shape=config.dim, eps=config.eps)
        
        
    def forward(self, hidden_states):

        def shape(x):
            """ separate heads """
            return x.view(1, -1, 12, 64).transpose(1, 2)

        def unshape(x):
            """ group heads """
            return x.transpose(1, 2).contiguous().view(1, -1, 12 * 64)

        bs=hidden_states.shape[0]
        n_nodes= hidden_states.shape[1] 

        query=key=value=hidden_states
        q = self.q_lin(query)
        k = self.k_lin(key)
        v = self.v_lin(value)

        # Separating the heads
        q = shape(q)  # (bs, n_heads, q_length, dim_per_head)
        k = shape(k)  # (bs, n_heads, k_length, dim_per_head)
        v = shape(v)  # (bs, n_heads, k_length, dim_per_head)
        
        # Normalizing the query-tensor
        q = q / math.sqrt(q.shape[-1]) 

        
        # 2. START YOUR CODE HERE #
        
        # Compute attention scores 
        attention_score = torch.matmul(q, k.transpose(-2,-1))
       
        # Transform the scores into probability distribution via softmax
        attention_weights = torch.softmax(attention_score, dim=-1)
        
        # Compute the weighted representation of the value-tensor (aka context)
        context = torch.matmul(attention_weights, v)
        
        # 2. END YOUR CODE HERE #
        
        # Merging the heads again 
        context = unshape(context)  # (bs, q_length, dim)

        # Additional projection of the context to get the output of the self-attention block
        sa_output = self.out_lin(context) 
        sa_output = self.sa_layer_norm(sa_output + hidden_states)

        # Feed-forward network to compute the attention block output
        x = self.lin1(sa_output)
        x = nn.functional.gelu(x)
        ffn_output = self.lin2(x)
        ffn_output = self.output_layer_norm(ffn_output + sa_output)

        return  ffn_output, context 
      
block = AttentionBlock(config)
# Test if your attention block computes an output
block_output = block(embeddings)
block_output

(tensor([[[ 0.7114, -0.0722, -0.5230,  ..., -0.1903, -0.7819, -0.5930],
          [ 0.2216, -0.9576,  0.9970,  ..., -0.0926, -1.1536,  0.8901],
          [ 0.4497, -0.6879,  1.1513,  ...,  0.0209, -0.9052,  1.1951],
          ...,
          [-0.1305, -0.3480,  0.5797,  ...,  0.3765, -0.8779,  0.5787],
          [ 0.6187,  0.2542,  0.1652,  ...,  1.0826, -1.0824,  0.9224],
          [-0.1873,  0.0490,  0.7358,  ..., -1.2114, -0.6134, -0.7558]]],
        grad_fn=<NativeLayerNormBackward0>),
 tensor([[[ 0.1446, -0.2007,  0.1056,  ..., -0.1175, -0.1715,  0.0907],
          [ 0.1336, -0.2000,  0.0524,  ..., -0.0759, -0.1545,  0.1869],
          [ 0.0561, -0.2531,  0.2032,  ..., -0.1044, -0.1423,  0.1895],
          ...,
          [ 0.1222, -0.1546,  0.1088,  ..., -0.0707, -0.1273,  0.2081],
          [ 0.1310, -0.1742,  0.0943,  ..., -0.1120, -0.2209,  0.0517],
          [ 0.1160, -0.2207,  0.0843,  ..., -0.1133, -0.2249,  0.0621]]],
        grad_fn=<ViewBackward0>))

# 3. Building the model (15 points)

Now, we can finally put it all together. For this, please, write the following missing code:


1.   \_\_init\_\_: Add the attention layers (the encoder) to the model.
2.   forward: Add the missing code for sequentially looping through the attention layers.
3. forward: Add the classifier, which consists of [1. pre_classifier, 2. ReLU activation, 3. classifier] and eventually returns the logit scores.




In [ ]:
torch.manual_seed(0)

class DistillBertAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.n_layers=config.n_layers

        # embedding
        self.embeddings = Embeddings(config)
        
        # encoder
        # 1. START YOUR CODE HERE #
        self.encoder = AttentionBlock(config)
        # 1. END YOUR CODE HERE #
        
        # classification
        self.pre_classifier =  nn.Linear(in_features=config.dim, out_features=config.dim, bias=True)
        self.classifier =  nn.Linear(in_features=config.dim, out_features=config.n_classes, bias=True)

        self.attention_probs = {i: [] for i in range(config.n_layers)}

    def forward(self, input_ids):
        """
        Parameters:
            input_ids (torch.Tensor): torch.tensor(bs, max_seq_length) The token ids to embed.
        Returns: torch.tensor(bs, n_classes) The computed logit scores for each class.
        """

        # Computing the embeddings
        hidden_states =  self.embeddings(input_ids=input_ids).to(self.config.device)

        # Iteratively going through the attention layers
        encoder_input = hidden_states
        # 2. START YOUR CODE HERE #
        encoder_block = self.encoder(encoder_input)
        
        
        for x in encoderin 

        # 2. END YOUR CODE HERE #
            
        # Pooling by selection the [CLS] token
        pooled_output = output[:, 0]  # (bs, dim)
        
        # Classification
        # 3. START YOUR CODE HERE #

        
        
        
        # 3. END YOUR CODE HERE #

        return logits

model = DistillBertAttention(config)
state_dict = torch.load('distilbert.pt')
_ = model.load_state_dict(state_dict)
_ = model.eval()

In [ ]:
# Predict your output
logits = model(inputs['input_ids'])

# 4. Visualize the attention weights (10 points)

Let's now look at what tokens the model selects in its self-attention blocks.

1.   Use the tokenizer to map the 'input_ids' back to 'tokens'.
2.   Extract attention probabilities for every layer by averaging over the attention heads in each layer. You should get a matrix of size [n_layers x seq_length x seq_length]
3. For each layer plot the resulting attention matrix. Hint: Use cmap='Reds' and vmin=0, vmax=1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# YOUR CODE HERE #
